In [2]:
import sqlite3
from pathlib import Path

__file__ = %pwd
DATA = Path(__file__).parent.parent / "data"
PRIORITIES = DATA / "Priorities"
DATABASE = DATA / "databases" / "exploration.db"

con = sqlite3.connect(DATABASE)
cur = con.cursor()

In [ ]:
import pandas as pd

# Load long-format tables where ReportingCategory exists as a column
census_day = pd.read_sql_query("SELECT * FROM CensusDay", con)
acgr = pd.read_sql_query("SELECT * FROM ACGR", con)

# Join on SchoolCode and AcademicYear/Year
merged = pd.merge(
    census_day, 
    acgr,
    left_on=['SchoolCode', 'AcademicYear'],
    right_on=['SchoolCode', 'Year'],
    how='inner'
)


merged